In [1]:
import torch
import argparse
import time
import math
import os, sys
import itertools
import pickle
from tqdm import tqdm
import numpy as np
import os.path as osp

In [2]:
torch.cuda.device_count()

1

In [3]:
def loadpkl(path):
    with open(path, 'rb') as f:
        rst=pickle.load(f)
    return rst

In [4]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/full_id_abs_title_cite_ref_author.pkl"
full_data=loadpkl(path)

In [7]:
path="/share/data/mei-work/kangrui/github/ssref/result/eval_rerank/test_result/eval_key_id2embed.pkl"
key_embedding=loadpkl(path)

In [5]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_test_pred_1000.pkl"
pred_set=loadpkl(path)

In [6]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_test_gold.pkl"
gold_set=loadpkl(path)

In [8]:
query_id2embedding=key_embedding
key_id2embedding=key_embedding

In [9]:
candidate_set={}
candidate_ref_set={}
for k,v in pred_set.items():
    cur=set()
    for paper in v[:200]:
        cur.add(paper)
        refset=set()
        for ref in full_data[paper]["references"]:
            cur.add(ref)
            refset.add(ref)
        candidate_ref_set[paper]=refset
    candidate_set[k]=list(cur)

In [17]:
def rerank_weighted_mean(full_data,candidate_set,query_id2embedding,key_id2embedding,candidate_ref_set,rerank_num=None,device=None):
    cnt=0
    pbar = tqdm(range(0,len(candidate_set),1), postfix=f"calculating f1")
    query=list(candidate_set.keys())
    empty_embed=torch.tensor(np.zeros_like(next(iter(key_id2embedding.values()))))
    empty_embed_cnt=0
    if rerank_num is None:
        for v in candidate_set.values():
            if rerank_num is None or rerank_num>len(v):
                rerank_num=len(v)
                
    rerank_candidate={}   
    for i in pbar:
        query_paper=query[i]
        query_embedding=torch.tensor(query_id2embedding[query_paper]).unsqueeze(0)
        
        cited_dict={}
        for candidate in candidate_set[query_paper]:
            
            if candidate in candidate_ref_set.keys():
                # this is possibly the anchor paper
                for ref in list(candidate_ref_set[candidate]):
                    # deal with the ref of an anchor paper
                    if ref in candidate_set[query_paper]:
                        if ref not in cited_dict.keys():
                            cited_dict[ref]=[candidate]
                        else:
                            cited_dict[ref].append(candidate)
                    else:
                        # this is not anchor paper for this query
                        break
            if candidate not in cited_dict.keys():
                cited_dict[candidate]=[candidate]
            else:
                cited_dict[candidate].append(candidate)
                
                
        key_embedding=[]
        for candidate in candidate_set[query_paper]:
            candidate_embedding=[]
            total_weight=1
            for paper in cited_dict[candidate]:
                if paper in key_id2embedding:
                    weight=full_data[paper]['citationCount']
                    total_weight+= weight
                    candidate_embedding.append(weight*torch.tensor(key_id2embedding[paper]))
            if len(candidate_embedding)==0:
                candidate_embedding.append(empty_embed)
                empty_embed_cnt+=1
            candidate_embedding=torch.sum(torch.stack(candidate_embedding),dim=0)/total_weight
            
            key_embedding.append(candidate_embedding)
        
        
        key_embedding=torch.stack(key_embedding)
        query_embedding=query_embedding.to(device)
        key_embedding=key_embedding.to(device)
        pred_logits=torch.mm(query_embedding,key_embedding.T)
        top_k=torch.topk(pred_logits[0],k=rerank_num)[1]
#         print(pred_logits[0][top_k[-5:]])
        rerank=[candidate_set[query_paper][idx.cpu().item()] for idx in top_k]
        rerank_candidate[query_paper]=rerank
    print("empty embedding cnt->",empty_embed_cnt)
    print("min rerank num->",rerank_num)
    return rerank_candidate


        


In [25]:
def rerank_weighted_mean(full_data,candidate_set,query_id2embedding,key_id2embedding,candidate_ref_set,rerank_num=None,device=None):
    cnt=0
    pbar = tqdm(range(0,len(candidate_set),1), postfix=f"calculating f1")
    query=list(candidate_set.keys())
    empty_embed=torch.tensor(np.zeros_like(next(iter(key_id2embedding.values()))))
    empty_embed_cnt=0
    if rerank_num is None:
        for v in candidate_set.values():
            if rerank_num is None or rerank_num>len(v):
                rerank_num=len(v)
                
    rerank_candidate={}   
    for i in pbar:
        query_paper=query[i]
        query_embedding=torch.tensor(query_id2embedding[query_paper]).unsqueeze(0)
        
        cited_dict={}
        for candidate in candidate_set[query_paper]:
            
            if candidate in candidate_ref_set.keys():
                # this is possibly the anchor paper
                for ref in list(candidate_ref_set[candidate]):
                    # deal with the ref of an anchor paper
                    if ref in candidate_set[query_paper]:
                        if ref not in cited_dict.keys():
                            cited_dict[ref]=[candidate]
                        else:
                            cited_dict[ref].append(candidate)
                    else:
                        # this is not anchor paper for this query
                        break
            if candidate not in cited_dict.keys():
                cited_dict[candidate]=[candidate]
            else:
                cited_dict[candidate].append(candidate)
                
                
        key_embedding=[]
        for candidate in candidate_set[query_paper]:
            candidate_embedding=[]
            total_weight=1
            for paper in cited_dict[candidate]:
                if paper in key_id2embedding:
                    weight=full_data[paper]['citationCount']
                    total_weight+= weight
                    candidate_embedding.append(weight*torch.tensor(key_id2embedding[paper]))
            if len(candidate_embedding)==0:
                candidate_embedding.append(empty_embed)
                empty_embed_cnt+=1
            candidate_embedding=torch.sum(torch.stack(candidate_embedding),dim=0)/total_weight

            key_embedding.append(candidate_embedding)
        
        
        key_embedding=torch.stack(key_embedding)
        query_embedding=query_embedding.to(device)
        key_embedding=key_embedding.to(device)
        pred_logits=torch.mm(query_embedding,key_embedding.T)
        top_k=torch.topk(pred_logits[0],k=rerank_num)[1]
#         print(pred_logits[0][top_k[-5:]])
        rerank=[candidate_set[query_paper][idx.cpu().item()] for idx in top_k]
        rerank_candidate[query_paper]=rerank
    print("empty embedding cnt->",empty_embed_cnt)
    print("min rerank num->",rerank_num)
    return rerank_candidate



In [26]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
rerank_candidate=rerank_weighted_mean(full_data,candidate_set,query_id2embedding,key_id2embedding,candidate_ref_set,device=device)

In [20]:
sys.path.append("/share/data/mei-work/kangrui/github/ssref")
from utils.test_utils import *
from utils.utils import *
def show_rerank_rst(rerank_candidate,gold_set):
    pool_length=len(next(iter(rerank_candidate.values())))
    i=2
    while 1:
        all_pred_topN=[]
        all_gold=[]
        for query_paper,paperlist in rerank_candidate.items():
            topN=paperlist[:i]
            all_pred_topN.append(topN)
            all_gold.append(gold_set[query_paper])
        all_pred_topN=np.array(all_pred_topN)
        metric = get_precision_recall_f1(all_pred_topN, all_gold)
        print(f"top{i}:\n{metric}")
        if i>pool_length:
            break
        i*=2

In [21]:
show_rerank_rst(rerank_candidate,gold_set)

top2:
{'precision': 0.1027542372881356, 'recall': 0.005046826222684703, 'f1': 0.009621106923229516, 'true_pos': 194, 'false_pos': 1694, 'false_neg': 38246, 'num_sample': 944}
top4:
{'precision': 0.08501059322033898, 'recall': 0.008350676378772112, 'f1': 0.015207504263786241, 'true_pos': 321, 'false_pos': 3455, 'false_neg': 38119, 'num_sample': 944}
top8:
{'precision': 0.0791843220338983, 'recall': 0.015556711758584807, 'f1': 0.026004522525656635, 'true_pos': 598, 'false_pos': 6954, 'false_neg': 37842, 'num_sample': 944}
top16:
{'precision': 0.06951800847457627, 'recall': 0.027315296566077004, 'f1': 0.03922008068130883, 'true_pos': 1050, 'false_pos': 14054, 'false_neg': 37390, 'num_sample': 944}
top32:
{'precision': 0.06173861228813559, 'recall': 0.04851716961498439, 'f1': 0.05433515907236918, 'true_pos': 1865, 'false_pos': 28343, 'false_neg': 36575, 'num_sample': 944}
top64:
{'precision': 0.052932997881355935, 'recall': 0.0831945889698231, 'f1': 0.0647001699441612, 'true_pos': 3198, 'f